### Introduction

The dataset consists of mails sent between Enron employees. 
- Each folder in the maildir directory represents a user.

#### Importing

In [2]:
import tarfile
import os
import networkx as nx
import re

In [ ]:
path = "D:\\pythondata\\email\\enron_mail_20150507.tar"

In [ ]:
# extract the contents of the tarred file and save the to our pwd
tarfile.open(path, "r").extractall()

In [ ]:
print("Number of Users: ", len(os.listdir("maildir")))
user_list = os.listdir("maildir")
c = 0

for user in user_list:
    if "sent" or "sent_items" in os.listdir("maildir" + "\\" + user):
        c += 1
        
print(c)
    # sent_items

### Build graph

In [ ]:
g = nx.Graph()

# loop through all of the users
for j, user in enumerate(user_list[:10]):
    
    user_dir = os.path.join("maildir", user)
    
    # if the user has a "sent_items" order use sent_items, otherwiese use the "sent" directory
    if "sent_items" in os.listdir(user_dir):
        path = os.path.join("maildir", user, "sent_items")
    elif "sent" in os.listdir(user_dir):
        path = os.path.join("maildir", user, "sent")
        
    # file names of all the mails the user sent.
    mails = os.listdir(path)
    
    for i, mail in enumerate(mails):
        with open(os.path.join(path, mail)) as f:
            mail_content = " ".join(f.read().split())
            to = re.findall(r"To: (.+?) Subject:", mail_content)
            
            # add the users email to the node set.
            if i == 0:
                user_email = re.findall(r"From: ([a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}\b)", mail_content)[0]
                g.add_node(user_email, company="enron")
                print(f"Looking at connections of node: {user_email} Node {j} out of {c}")
            
            if to != []:
                for i in to[0].split(","):  
                    # confirm that the supposed target node really is an email address.
                    if re.findall(r"[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}\b", i) != []:
                        
                        # if the email address has the @enron.com we can confirm that the employees is at enron.
                        if re.findall(r"@enron\.com", i):
                            g.add_node(i, company="enron")
                            g.add_edge(user_email, i)
                        else:
                            g.add_node(i, company="not_enron")
                            g.add_edge(user_email, i)

In [32]:
len(g.nodes())

1718

In [33]:
len(nx.get_node_attributes(g, "company"))

1718

In [ ]:
nx.draw(g, node_size=10)

In [9]:
nlp = spacy.load("en_core_web_md")
matcher = Matcher(nlp.vocab)
pattern = [{"LIKE_EMAIL": True}]

matcher.add("EMAIL_ADRESS", [pattern])

doc = nlp(mail_content)

In [10]:
# gives back a tuple for each match. The first element of a lexeme, the next the start token and the last element the end token.
matches = matcher(doc)
matches

[(2197859665807148658, 23, 24), (2197859665807148658, 27, 28)]

In [ ]:
for ent in doc.ents:
    print(ent.label_, ent.text)

In [ ]:
doc

In [6]:
for match in matches:
    print(nlp.vocab[match[0]].text)

EMAIL_ADRESS
EMAIL_ADRESS


In [ ]:
for i in doc.sents:
    print(i)

look at Doc2Vec and

Similar People

Predict position in company

Community Detection